In [1]:
# @title Imports

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

import ipdb

import optuna

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder

%load_ext autotime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
seq = ['seq_'+str(i) for i in range(100)]
X_train=pd.read_csv('./data/Xtr_mat100.csv', sep=' ', names = seq) #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte_mat100.csv', sep=' ', names = seq) #we will use this data set later to validate our model

Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model

time: 165 ms


In [3]:
# X_train=pd.read_csv('./data/train_data_preprocessing1.csv', sep=',') #we use this dataset to train our model
# Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
# X_test=pd.read_csv('./data/test_data_preprocessing1.csv', sep=',')

time: 880 µs


In [4]:
# X_test.head()

time: 640 µs


In [5]:
X_train.tail()

,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,seq_8,seq_9,seq_10,seq_11,seq_12,seq_13,seq_14,seq_15,seq_16,seq_17,seq_18,seq_19,seq_20,seq_21,seq_22,seq_23,seq_24,seq_25,seq_26,seq_27,seq_28,seq_29,seq_30,seq_31,seq_32,seq_33,seq_34,seq_35,seq_36,seq_37,seq_38,seq_39,...,seq_60,seq_61,seq_62,seq_63,seq_64,seq_65,seq_66,seq_67,seq_68,seq_69,seq_70,seq_71,seq_72,seq_73,seq_74,seq_75,seq_76,seq_77,seq_78,seq_79,seq_80,seq_81,seq_82,seq_83,seq_84,seq_85,seq_86,seq_87,seq_88,seq_89,seq_90,seq_91,seq_92,seq_93,seq_94,seq_95,seq_96,seq_97,seq_98,seq_99
1995,0.000000,0.021739,0.01087,0.01087,0.01087,0.000000,0.01087,0.01087,0.021739,0.000000,0.000000,0.043478,0.000000,0.000000,0.00000,0.032609,0.000000,0.01087,0.010870,0.010870,0.000000,0.010870,0.000000,0.00000,0.021739,0.032609,0.010870,0.010870,0.021739,0.00000,0.000000,0.021739,0.010870,0.00000,0.000000,0.00000,0.021739,0.010870,0.010870,0.00000,...,0.021739,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.021739,0.00000,0.010870,0.010870,0.021739,0.021739,0.000000,0.01087,0.010870,0.01087,0.01087,0.010870,0.010870,0.043478,0.000000,0.021739,0.00000,0.00000,0.000000,0.000000,0.000000,0.032609,0.010870,0.021739,0.01087,0.021739,0.021739,0.010870,0.000000,0.032609,0.01087,0.010870
1996,0.010870,0.032609,0.00000,0.01087,0.00000,0.043478,0.00000,0.00000,0.010870,0.021739,0.032609,0.000000,0.043478,0.054348,0.00000,0.000000,0.021739,0.00000,0.010870,0.010870,0.010870,0.000000,0.010870,0.00000,0.010870,0.000000,0.000000,0.000000,0.000000,0.01087,0.043478,0.000000,0.000000,0.01087,0.000000,0.00000,0.000000,0.032609,0.010870,0.01087,...,0.000000,0.01087,0.000000,0.021739,0.000000,0.021739,0.000000,0.00000,0.000000,0.01087,0.010870,0.000000,0.000000,0.010870,0.032609,0.01087,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000,0.00000,0.01087,0.032609,0.021739,0.054348,0.000000,0.000000,0.032609,0.00000,0.021739,0.000000,0.010870,0.000000,0.000000,0.00000,0.010870
1997,0.000000,0.000000,0.00000,0.00000,0.01087,0.000000,0.01087,0.00000,0.000000,0.000000,0.021739,0.010870,0.032609,0.000000,0.00000,0.010870,0.000000,0.00000,0.010870,0.000000,0.000000,0.000000,0.010870,0.01087,0.010870,0.000000,0.000000,0.021739,0.000000,0.00000,0.010870,0.010870,0.010870,0.01087,0.043478,0.01087,0.000000,0.010870,0.021739,0.00000,...,0.000000,0.00000,0.021739,0.000000,0.021739,0.010870,0.021739,0.01087,0.010870,0.00000,0.010870,0.010870,0.021739,0.010870,0.010870,0.01087,0.032609,0.01087,0.01087,0.010870,0.032609,0.000000,0.021739,0.010870,0.00000,0.01087,0.000000,0.010870,0.000000,0.054348,0.021739,0.021739,0.00000,0.010870,0.010870,0.010870,0.000000,0.010870,0.01087,0.032609
1998,0.021739,0.032609,0.00000,0.01087,0.00000,0.010870,0.00000,0.01087,0.021739,0.010870,0.010870,0.021739,0.010870,0.010870,0.01087,0.000000,0.000000,0.00000,0.021739,0.010870,0.021739,0.010870,0.021739,0.01087,0.000000,0.021739,0.000000,0.000000,0.000000,0.00000,0.021739,0.021739,0.021739,0.01087,0.010870,0.00000,0.000000,0.021739,0.010870,0.00000,...,0.000000,0.01087,0.000000,0.010870,0.000000,0.000000,0.010870,0.00000,0.000000,0.01087,0.000000,0.032609,0.000000,0.000000,0.010870,0.00000,0.010870,0.01087,0.00000,0.021739,0.010870,0.010870,0.021739,0.000000,0.01087,0.01087,0.010870,0.010870,0.021739,0.010870,0.000000,0.000000,0.00000,0.021739,0.021739,0.021739,0.021739,0.010870,0.00000,0.000000
1999,0.000000,0.010870,0.00000,0.01087,0.00000,0.000000,0.00000,0.00000,0.021739,0.010870,0.010870,0.043478,0.000000,0.000000,0.00000,0.010870,0.000000,0.00000,0.032609,0.021739,0.000000,0.021739,0.010870,0.01087,0.010870,0.010870,0.043478,0.000000,0.021739,0.00000,0.000000,0.021739,0.021739,0.01087,0.010870,0.01087,0.043478,0.010870,0.032609,0.01087,...,0.021739,0.01087,0.010870,0.000000,0.010870,0.021739,0.000000,0.00000,0.010870,0.01087,0.021739,0.000000,0.021739,0.000000,0.000000,0.01087,0.010870,0.01087,0.00000,0.010870,0.021739,0.000000,0.010870,0.010870,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000

time: 216 ms


In [6]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 100)
The shape of the Y_train dataset is: (2000, 2)
time: 1.64 ms


In [7]:
# X_train['len'] = X_train.seq.apply(lambda x : len(x))

time: 361 µs


In [8]:
X_train.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


time: 9.17 ms


In [9]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

time: 709 µs


In [10]:
#  X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])
for i in range(0, 101, 1):
    X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])
    
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]]

#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+3])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+3])



time: 277 ms


In [11]:
X_train.head()

,Id,seq,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,seq_8,seq_9,seq_10,seq_11,seq_12,seq_13,seq_14,seq_15,seq_16,seq_17,seq_18,seq_19,seq_20,seq_21,seq_22,seq_23,seq_24,seq_25,seq_26,seq_27,seq_28,seq_29,seq_30,seq_31,seq_32,seq_33,seq_34,seq_35,seq_36,seq_37,...,seq_61,seq_62,seq_63,seq_64,seq_65,seq_66,seq_67,seq_68,seq_69,seq_70,seq_71,seq_72,seq_73,seq_74,seq_75,seq_76,seq_77,seq_78,seq_79,seq_80,seq_81,seq_82,seq_83,seq_84,seq_85,seq_86,seq_87,seq_88,seq_89,seq_90,seq_91,seq_92,seq_93,seq_94,seq_95,seq_96,seq_97,seq_98,seq_99,seq_100
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...,7,1,7,7,7,7,3,20,7,7,7,7,1,7,7,7,7,7,3,20,7,7,3,3,3,1,7,1,7,7,3,1,3,3,1,7,1,3,...,20,7,7,7,7,3,20,7,3,3,3,20,7,3,3,1,3,3,20,7,3,20,7,7,3,3,7,3,20,3,3,20,7,7,20,7,7,3,1,7
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...,3,7,7,3,3,20,7,7,7,7,7,3,3,1,3,1,20,7,20,7,1,7,20,7,3,20,20,1,3,3,20,7,20,7,20,7,7,7,...,20,3,1,7,7,3,1,7,3,1,7,3,20,7,7,7,20,1,3,20,1,1,1,7,7,7,7,3,3,20,7,20,3,3,1,7,7,20,7,7
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...,7,1,3,1,1,3,7,3,3,7,3,20,7,20,3,1,7,3,3,7,3,3,20,20,3,7,1,3,20,3,1,3,3,20,7,7,7,1,...,20,1,3,3,20,7,7,7,1,1,20,3,20,7,7,7,3,1,20,3,3,3,7,7,7,3,20,3,3,3,3,7,1,1,7,20,7,3,7,20
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...,7,3,3,20,3,3,3,20,20,7,7,3,1,3,3,1,3,7,7,7,1,7,1,3,3,1,7,20,20,20,20,7,7,1,7,7,7,7,...,3,1,3,20,7,20,3,1,7,7,1,1,7,7,3,20,7,1,1,7,20,20,20,3,7,7,7,7,3,1,7,1,7,20,7,3,20,1,1,1
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...,7,3,1,3,20,1,3,20,1,3,1,3,3,3,1,20,20,7,3,20,7,20,1,1,20,1,7,20,1,1,7,20,7,3,3,7,7,20,...,7,20,3,3,20,7,7,7,1,1,1,20,1,3,1,3,1,3,20,3,1,3,1,3,20,20,7,20,7,20,1,3,20,7,20,20,1,3,3,3


time: 110 ms


## What about counting number of different caracters 

In [12]:
# A, C, G, T

time: 520 µs


In [12]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',')

def spectrum_kernal(X_train, y, X_test, n=3, encoder=4, one_hot = True, normalise = False):
    
    d = {'A': 0.3, 'C':0.4, 'G':0.5, 'T':0.6}
    
    for i in range(0, 101-n+1, 1):
        X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+n])
        X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+n])
        
        X_train['seq_'+str(i)] = X_train['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        X_test['seq_'+str(i)] = X_test['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        
        
        
    X = X_train.drop(['seq', 'Id'], axis=1)
    X_t = X_test.drop(['seq', 'Id'], axis=1)
    y = Y_train.Bound
    
#     print(f'Train: \n{X.tail()}\n -----------------------\n')
#     print(f'Test: \n {X_t.tail()}')

    if one_hot:
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore')

        X_cross = X.values
        X_t = X_t.values
        
        enc = onehot_encoder.fit(X)
        X_cross = enc.transform(X)
        X_t_enc = enc.transform(X_t)
        
    elif normalise:
        scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
        scaler.fit(X)
        
        X_cross = scaler.transform(X)
        X_t_enc = scaler.transform(X_t)
        
    else :
        
        X_cross = X.values
        X_t_enc = X_t.values
    
    y_cross = y.values
    
    return X_cross, y_cross, X_t_enc

X_cross, y_cross, X_t_enc = spectrum_kernal(X_train, Y_train, X_test, n=5, encoder=3, one_hot = True, normalise = False)

time: 3.81 s


In [13]:
X_cross.shape

(2000, 35014)

time: 2.85 ms


In [14]:
X_t_enc.shape

(1000, 35014)

time: 2.6 ms


In [13]:
# X_train.seq.apply(lambda x : np.unique(x[:2]))


time: 426 µs


In [17]:
X = X_train
X_t = X_test
y = Y_train.Bound

time: 1.88 ms


## Models

In [2]:
# class LogisticRegressionBinary():
#     def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
#         self.lr = lr
#         self.num_iter = num_iter
#         self.batch_size = batch_size
#         self.verbose = verbose
    
#     def __add_intercept(self, X):
#         intercept = np.ones((X.shape[0], 1))
#         return np.concatenate((intercept, X), axis=1)
    
#     def __sigmoid_func(self, z):
#         return 1 / (1 + np.exp(-z))
    
#     def __loss(self, h, y):
#         return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
#     def fit(self, X, y):
#         y = self.trans_y(y)
#         X = self.__add_intercept(X)
#         self.theta = np.zeros(X.shape[1])
        
#         for i in range(self.num_iter):
#             z = np.dot(X, self.theta)
#             h = self.__sigmoid_func(z)
                        
#             rand = np.random.choice(y.size, self.batch_size).squeeze()
#             gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
#             self.theta -= self.lr * gradient
#             #print('theta and grad',self.theta.shape ,  gradient.shape )
#             if(self.verbose == True and i % 100 == 0):
#                 z = np.dot(X, self.theta)
#                 h = self.__sigmoid(z)
#                 print(f'loss: {self.__loss(h, y)} \t')
    
#     def predict_probability(self, X):
#         X = self.__add_intercept(X)
    
#         return self.__sigmoid_func(np.dot(X, self.theta))
    
#     def predict(self, X, threshold=.5):
#           return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
#     def Accuracy_check(self,X,y):
#         return np.mean(self.predict(X)==y)
    
#     def trans_y(self, y):
#         if isinstance(y, pd.Series):
#             y = y.values
#         if isinstance(y, list):
#             y = np.array(y)
#         return y

time: 16.9 ms


In [3]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 453 ms


# Kernel

In [17]:
import numpy as np

### Functions for you to fill in ###

def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix


def rbf_kernel_2(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    n, d = X.shape
    m = Y.shape[0]
    
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
    
    return kernel_matrix


def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def laplace(X1, X2, alpha=10):
    return np.exp(-alpha*np.abs(X1-X2))

# def polynomial(X1, X2, d=2):
#     return (X1.dot(X2.T) +1)**d

def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def quadratic_kernel(X1, X2, power=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the quadratic kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (1 + linear_kernel(X1, X2))**power

def rbf_poly_kernel(X1, X2, sigma=10, d=20):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    
    return K+(X1.dot(X2.T) +1)**d

time: 163 ms


In [5]:
class ksolveRR_2():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
        self.sample_weights = sample_weights
            
    
    def fit(self):
        if self.sample_weights is not None:
            self.X *= self.sample_weights[:, None]
        
        X = self.X
        y = self.y
        lam = self.lam
        
        n, p = X.shape
        assert (len(y) == n)
        
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 69.3 ms


In [6]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
            
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 57.9 ms


In [7]:
import ipdb
# Logistic Ridge Regression (LRR)
class ksolveLRR():
    def __init__(self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        
        self.kernel = kernel
        
        self.sigma = sigma
        self.max_iter = max_iter
        self.tol = tol
        
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)

        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        K = self.kernel(X, X, self.sigma)

        # Initialize
        alpha = np.zeros(n)
        
        # Hint: Use IRLS
        for n_iter in range(self.max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*(f))
            
            alpha = ksolveRR_2(X, y, lam = 2*self.lam, \
                               sigma=self.sigma, sample_weights = w).fit().alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < self.tol:
                break  
                
                
        self.n_iter = n_iter
        self.alpha = alpha
        
        return self
    
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 108 ms


In [8]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'

import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

# SVM primal soft
class KernelSVM():
    def __init__(self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.C = C
        self.kernel = kernel
        self.lam = lam        
        self.sigma = sigma
        self.tol = tol
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        K = self.kernel(X, X, self.sigma)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
       # Compute support vectors and bias b
        sv = np.logical_and((self.alpha>self.tol), (self.C - self.alpha > self.tol))
        self.bias = y[sv] - K[sv].dot(self.alpha*y)
        self.bias =  self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
        
        
    def predict(self, X, threshold):
#         y_pred = self.kernel(X, self.X_).dot(self.alphas* seld.y_)
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where((K_x.dot(self.alpha * self.y) +  self.bias) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 370 ms


In [9]:
# # Logistic Ridge Regression (LRR)
# class perceptron():
#     def __init__(self, X, y, iteration=100):
#         self.theta = np.zeros(len(X))
#         self.X = X
#         self.y = y
#         self.iteration = iteration
#         self.theta_0 = 0
    
#     def fit(self):
        
#         X = self.X
#         y = self.y
        
#         n, p = X.shape
#         assert (len(y) == n)
        
#         for it in range(self.iteration):
#             for i in range(len(y)):
#                 if y[i]*(np.dot(self.theta, X[i]) + self.theta_0)<= 0 :
#                     self.theta = self.theta + (np.dot(y[i], X[i]))
#                     self.theta_0 += y[i]
    
        
#     def predict(self, X, threshold):
#         return np.where(X.dot(self.self.theta)+self.theta_0 >= threshold, 1, 0)
        
          
#     def Accuracy_check(self,X, y, threshold=.5):
#         return np.mean(self.predict(X, threshold)==y)

time: 5.34 ms


# Cross Validation

In [10]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TransformerM
# from sklearn.feature_extraction.text import TfidfTransformer

time: 47.3 ms


In [11]:
# vectorizer = CountVectorizer(max_features=2000)
# vectorizer.fit_transform(X[:10])
# # vectorizer.get_feature_names()

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

time: 2.02 ms


In [12]:
kfold=KFold(n_splits=5)
# onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# # X_cross = X.values
# # X_t = X_t.values

# y_cross = y.values

# # vectorizer = TfidfVectorizer()
# # X_cross = vectorizer.fit_transform(X)

# # X_cross = onehot_encoder.fit_transform(X.values[:, :101])
# # X_t_enc = onehot_encoder.fit_transform(X_t.values[:, :101])

# # X_cross = onehot_encoder.fit_transform(X)
# # X_t_enc = onehot_encoder.fit_transform(X_t)


# X_cross = X.values
# X_t_enc = X_t.values


# # scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
# # scaler.fit(X_cross)

# # X_cross = scaler.transform(X_cross)

time: 24.3 ms


In [13]:
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.feature_selection import SelectKBest, chi2

time: 1.13 ms


In [14]:
# def feature_selector(X):
#     #sel = VarianceThreshold()
#     sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#     sel.fit_transform(X)
#     return X
# def best_feature_selector(X,y,num_features=50):
#     #print(X.shape)
#     features = SelectKBest(chi2, k=num_features).fit(X, y)
#     #print(X_new.shape)
#     return features

time: 1.91 ms


In [15]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',')

def spectrum_kernal(X_train, y, X_test, n=3, encoder=4, one_hot = True, normalise = False):
    
    d = {'A': 0.3, 'C':0.4, 'G':0.5, 'T':0.6}
    
    for i in range(0, 101-n+1, 1):
        X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+n])
        X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+n])
        
        X_train['seq_'+str(i)] = X_train['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        X_test['seq_'+str(i)] = X_test['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        
        
        
    X = X_train.drop(['seq', 'Id'], axis=1)
    X_t = X_test.drop(['seq', 'Id'], axis=1)
    y = Y_train.Bound
    
#     print(f'Train: \n{X.tail()}\n -----------------------\n')
#     print(f'Test: \n {X_t.tail()}')

    if one_hot:
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore')

        X_cross = X.values
        X_t = X_t.values
        
        enc = onehot_encoder.fit(X)
        X_cross = enc.transform(X)
        X_t_enc = enc.transform(X_t)
        
    elif normalise:
        scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
        scaler.fit(X)
        
        X_cross = scaler.transform(X)
        X_t_enc = scaler.transform(X_t)
        
    else :
        
        X_cross = X.values
        X_t_enc = X_t.values
    
    y_cross = y.values
    
    return X_cross, y_cross, X_t_enc

# X_cross, y_cross, X_t_enc = spectrum_kernal(X_train, Y_train, X_test, n=5, encoder=3, one_hot = True, normalise = False)

time: 215 ms


In [33]:
6e+0

6.0

time: 3.25 ms


In [20]:
X_train_ = pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train_ = pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test_ = pd.read_csv('./data/Xte.csv', sep=',')


def objective_sgd(trial):
    
    
#     c  = trial.suggest_loguniform('c', 1e-20, 2e1)
    sigma  = trial.suggest_loguniform('sigma', 1e-0, 1e+1) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
#     kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
    lam = trial.suggest_loguniform('lam', 1e-18, 1e-0)
#     tol = trial.suggest_loguniform('tol', 1e-7, 1e-0)
    
#     normalise = trial.suggest_categorical('normalise', [False , True])
    
    n = trial.suggest_int('n', 1, 3)
    
    tol = 1e-7
#     model = trial.suggest_categorical('models', [ksolveRR , ksolveRR_2, ksolveLRR, KernelSVM])
    
    # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
    # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
    # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
    # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
#     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
#               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
#     
    kenel = rbf_kernel
    models = {ksolveLRR: 'k Logistic Ridge Reg'} 
      
#     accuracy = []
#     for model in models:
#     ipdb.set_trace()
    
#     X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
#     Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
#     X_test=pd.read_csv('./data/Xte.csv', sep=',')
    X_cross, y_cross, X_t_enc = spectrum_kernal(X_train_, Y_train_, X_test_, n=n, encoder=4, one_hot = True, normalise = False)
    
    for model in models:
        accuracy = []
        for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
            X_train, y_train = X_cross[train_index], y_cross[train_index]
            X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]


    #             ipdb.set_trace()
            if models[model] == 'weigh Ridge Reg':
                sample_weights = np.random.rand(len(y_train))
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

            elif models[model] == 'k Logistic Ridge Reg':
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
            elif models[model] == 'k Ridge Reg':
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
            else:
                model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)

            model_curr.fit()
            accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
    #             print(f'accurracy fold {i}: {accuracy[i]}')

    #         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()

study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=300, show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in true_divide



[I 2020-05-29 19:05:16,042] Finished trial#0 with value: 0.501 with parameters: {'sigma': 1.6282182386299346, 'lam': 1.213724180016989e-15, 'n': 2}. Best is trial#0 with value: 0.501.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in true_divide



[I 2020-05-29 19:05:22,601] Finished trial#1 with value: 0.501 with parameters: {'sigma': 1.818510702977888, 'lam': 1.756258983651971e-13, 'n': 1}. Best is trial#0 with value: 0.501.


...

[I 2020-05-29 20:56:34,999] Finished trial#297 with value: 0.5 with parameters: {'sigma': 9.6909289501821, 'lam': 2.117179469689603e-18, 'n': 2}. Best is trial#256 with value: 0.5189999999999999.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in true_divide



[I 2020-05-29 20:57:02,695] Finished trial#298 with value: 0.484 with parameters: {'sigma': 8.858460880644042, 'lam': 2.3978796516392533e-17, 'n': 2}. Best is trial#256 with value: 0.5189999999999999.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in true_divide



[I 2020-05-29 20:57:30,595] Finished trial#299 with value: 0.505 with parameters: {'sigma': 8.328893839851037, 'lam': 5.617580793667513e-18, 'n': 2}. Best is trial#256 with value: 0.5189999999999999.

Accuracy: 0.5189999999999999
Best hyperparameters: {'sigma': 9.56291717324295, 'lam': 4.7329565237648745e-18, 'n': 2}
time: 1h 52min 23s


In [35]:
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.6605000000000001
Best hyperparameters: {'sigma': 4.537999983263391, 'lam': 2.005175225990806e-13, 'n': 2}
time: 1.59 ms


In [ ]:
# kenel = rbf_kernel
# models = {ksolveRR: 'k Ridge Reg'} 
    # Accuracy: 0.624
    # Best hyperparameters: {'sigma': 0.5676751743446526, 'lam': 5.181586550863371e-06}
    # time: 973 µs
    #------------------------
    # Accuracy: 0.6605000000000001
    # Best hyperparameters: {'sigma': 4.537999983263391, 'lam': 2.005175225990806e-13, 'n': 2}
    # time: 1h 12min 30s
    
# kenel = quadratic_kernel
# models = {ksolveRR: 'k Ridge Reg'} 
    # Accuracy: 0.6585
    # Best hyperparameters: {'sigma': 4.537999983263391, 'lam': 2.005175225990806e-13, 'n': 2}
    # time: 1h 27min 3s
    #------------------------


In [ ]:
# Augmented data

# sigma  = trial.suggest_loguniform('sigma', 1e-20, 20)
# lam = trial.suggest_loguniform('lam', 1e-20, 1e-1)

# models = {ksolveRR: 'k Ridge Reg'} (Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.065620491225982, 'lam': 0.013374107290659768}
    # time: 49min 21s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575 # 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.119788517147901, 'lam': 1.2752298700618223e-14}, {'sigma': 4.063158315715049, 'lam': 8.30834772639223e-05}
    # time: 32min 57s

    
# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)

# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.069738272304652, 'lam': 0.053817561640154984}
    # time: 48min 12s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.001185698777986, 'lam': 1.9770379950513775e-10}
    # time: 29min 58s
    


# Original data

# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
  
# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data standardize)
    # kenel = rbf_kernel
    # Accuracy: 0.612
    # Best hyperparameters: {'sigma': 2.8925324917718167, 'lam': 1.2575244169567934e-08}
    # time: 28min 33s 
    #----
    # kenel = quadratic_kernel
    # Accuracy: EROOR LinearAlg
    # Best hyperparameters: 
    # time: 

In [55]:
#     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
#               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'

# c  = 
sigma  = 4.119788517147901
kenel = rbf_kernel
lam = 1.2752298700618223e-14
# tol = 
    
#     ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
#     ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
#     ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
#     KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel

    
models = {ksolveRR : 'k Ridge Reg'}

for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=500, tol = tol, kernel = kenel)
        
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.5975
accurracy fold 1: 0.6625
accurracy fold 2: 0.68
accurracy fold 3: 0.6475
accurracy fold 4: 0.7

Average accuracy k Ridge Reg is : 0.6575

time: 1.07 s


In [56]:
# 0.6535 = 0.68800
# 0.657 = 0.69200

time: 525 µs


In [58]:
# # Cehckinf full model
# model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
# # model = svm_primal_soft_to_qp(X_cross, y_cross, C=1)

# model.fit()

# model.Accuracy_check(X_cross, y_cross, threshold=0.5)

time: 881 µs


# Predictions

In [59]:
model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

time: 393 ms


In [60]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


time: 8.73 ms


In [61]:
sample['Bound'] = y_pred

time: 1.42 ms


In [62]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,1
998,998,1
999,999,1


time: 7.8 ms


In [63]:
sample.to_csv('./ksolveRR_6575_cv_rbf_kernel_sigma_4.119788517147901_lam_1.2752298700618223e-14.csv', index=False)

time: 3.12 ms
